In [1]:
pwd

'E:\\Projects\\MLOPS\\Demo\\wineQulity_mlops_project\\notebooks'

In [4]:
os.chdir('E:\\Projects\\MLOPS\\Demo\\wineQulity_mlops_project')

In [3]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

In [15]:
def log_production_model(config_path):
    try:
        # Read configuration parameters
        config = read_params(config_path)
        mlflow_config = config["mlflow_config"]
        model_name = mlflow_config["registered_model_name"]
        remote_server_uri = mlflow_config["remote_server_uri"]

        # Set MLflow registry URI
        mlflow.set_registry_uri(remote_server_uri)

        # Search for runs in the experiment with ID 1
        runs = mlflow.search_runs(experiment_ids=1)

        # Find the run with the lowest MAE
        lowest = runs['metrics.mae'].sort_values(ascending=True)[0]
        lowest_run_id = runs[runs['metrics.mae'] == lowest]["run_id"][0]

        # Initialize the MLflow client
        client = mlflow.tracking.MlflowClient()

        # Loop through model versions
        for mv in client.search_model_versions(f"name='{model_name}'"):
            mv = dict(mv)
            current_version = mv["version"]

            if mv["run_id"] == lowest_run_id:
                # Transition the model version to "Production"
                logged_model = mv["source"]
                pprint(mv, indent=4)
                client.transition_model_version_stage(
                    name=model_name,
                    version=current_version,
                    stage="Production"
                )
            else:
                # Transition other model versions to "Staging"
                client.transition_model_version_stage(
                    name=model_name,
                    version=current_version,
                    stage="Staging"
                )

    except Exception as e:
        print(f"An error occurred: {str(e)}")

log_production_model("params.yaml")


An error occurred: object of type 'int' has no len()
